In [ ]:
import mlflow
from ultralytics import YOLO
import os

In [ ]:
MLFLOW_TRACKING_URI = "file://" + os.path.abspath("./mlruns") 
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [ ]:
EXPERIMENT_NAME = "YOLOv11_Military_Detection"
mlflow.set_experiment(EXPERIMENT_NAME)

In [ ]:
# --- Entraînement ---
# Démarre une session d'enregistrement MLflow
with mlflow.start_run() as run:
    print("MLflow Run ID:", run.info.run_id)
    mlflow.log_param("model_type", "YOLOv11m")
    mlflow.log_param("dataset", "custom_military_v1")
    
    # Charger un modèle pré-entraîné
    model_path = os.path.abspath('../server/models/best.onnx')
    model = YOLO(model_path)

    # Entraîner le modèle (exemple avec des paramètres)
    # Assurez-vous d'avoir un fichier data.yaml pour votre dataset
    results = model.train(data='../../dataset/samurai/data.yaml', epochs=50, imgsz=640, batch=8)
    
    # Enregistrer les hyperparamètres
    mlflow.log_param("epochs", 50)
    mlflow.log_param("imgsz", 640)
    mlflow.log_param("batch_size", 8)
    # Enregistrer les métriques de performance finales
    # results.metrics contient les métriques comme mAP50-95
    final_map = results.metrics['metrics/mAP50-95(B)']
    mlflow.log_metric("mAP50-95", final_map)
    
    # Enregistrer le modèle entraîné comme un "artefact"
    # MLflow va sauvegarder le dossier 'runs/detect/train'
    model_path = results.save_dir
    mlflow.log_artifacts(model_path, artifact_path="yolo_model")
    
    print(f"Modèle entraîné et sauvegardé. mAP: {final_map}")
    print(f"Pour voir les résultats, lancez 'mlflow ui' dans votre terminal.")